# Requerimiento
Elabora un programa ("count_orders_basket_size_by_delay_status.py") que construya una tabla que muestre el número de órdenes que existieron por cantidad de productos dentro de la orden y el tipo de retraso de las categorías delay_status. El resultado de este script deberá ser un tabla denominada “count_orders_basket_size_by_delay_status.csv.”

# 1. Importamos librerías





In [6]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [4]:
DATA_PATH = '/content/'
FILE_NAME = 'oilst_processed.csv'

# 2. Cargamos archivo

Validamos que exista el archivo

In [7]:
print(f'ruta del archivo: {DATA_PATH + FILE_NAME}')
os.path.exists(DATA_PATH + FILE_NAME)

ruta del archivo: /content/oilst_processed.csv


True

Creamos dataframe

In [8]:
df_oilst = pd.read_csv(DATA_PATH + FILE_NAME)
df_oilst.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,distance_distribution_center,year,...,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,abbreviation,state_name
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,29.84,2017,...,3149,sao paulo,SP,3149.0,-23.574809,-46.587471,sao paulo,SP,SP,São Paulo
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,25.53,2018,...,47813,barreiras,BA,47813.0,-12.169860,-44.988369,barreiras,BA,BA,Bahia
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,61.56,2018,...,75265,vianopolis,GO,75265.0,-16.746337,-48.514624,vianopolis,GO,GO,Goiás
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,60.30,2017,...,59296,sao goncalo do amarante,RN,59296.0,-5.767733,-35.275467,sao goncalo do amarante,RN,RN,Rio Grande do Norte
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,80.97,2018,...,9195,santo andre,SP,9195.0,-23.675037,-46.524784,santo andre,SP,SP,São Paulo


In [13]:
df_oilst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
 8   distance_distribution_center   96470 non-null  float64       
 9   year                           99441 non-null  int64         
 10  month                          99441 non-null  int64         
 11  quarter        

In [12]:
# Lista de columnas a interpretar como fecha
columns_dates=[
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
    ]

# Lectura del archivo csv
df_oilst = pd.read_csv(
    os.path.join(DATA_PATH, FILE_NAME),
    parse_dates=columns_dates
    )
df_oilst.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
 8   distance_distribution_center   96470 non-null  float64       
 9   year                           99441 non-null  int64         
 10  month                          99441 non-null  int64         
 11  quarter        

# 3. Procesamos para obtener lo requerido

Hacemos uso de crosstab para crear una tabla que nos muestre número de órdenes por cantidad de productos dentro de la orden y el tipo de retraso de la categoría delay_status

In [24]:
pd.crosstab(
    df_oilst['total_products'],
    df_oilst['delay_status'],
    margins=True
    ).sort_values(by='long_delay').tail(10)


delay_status,long_delay,on_time,short_delay,All
total_products,,,,
15.0,0,2,0,2
20.0,0,2,0,2
21.0,1,0,0,1
5.0,7,180,17,204
6.0,11,175,12,198
4.0,17,467,21,505
3.0,49,1225,48,1322
2.0,307,6903,306,7516
1.0,4773,79642,4448,88863


# 4. Exportamos la tabla al archivo "count_orders_basket_size_by_delay_status.csv"

In [26]:
df_oilst.to_csv(
    os.path.join(DATA_PATH, 'count_orders_basket_size_by_delay_status.csv'),
    index=False
    )
#

In [27]:
#comprobamos que exista el archivo
os.path.exists(os.path.join(DATA_PATH, 'count_orders_basket_size_by_delay_status.csv'))
#

True